In [ ]:
# Verify weviate-client is installed and the database is live and ready
import weaviate
client = weaviate.Client("http://localhost:8080")
assert client.is_live()
assert client.is_ready()
client.get_meta()

In [ ]:
client.query.get(class_name='Web_Node', properties='doc_id,_node_content').with_limit(5).do()

In [ ]:

where_filter = {
  "path": ["doc_id"],
  "operator": "Equal",
  "valueString": "https://www.janelia.org/people/harikrushnan-balasubramanian"
}

query_result = (
  client.query
  .get(class_name='Web_Node', properties="doc_id,_node_content,text").with_additional(["id", "vector","distance"])
  .with_where(where_filter)
  .do()
)

query_result


In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from llama_index.legacy import LangchainEmbedding

query = "Who is the director of Scientific Computing?"


embed_model = LangchainEmbedding(OpenAIEmbeddings())
vector = embed_model.get_query_embedding(query)
class_name = 'Test_Node'
query_result = (
    client.query
    .get(class_name=class_name, properties="doc_id,extra_info,text")
    .with_additional("distance")
    .with_near_vector({"vector": vector})
    .with_limit(50)
    .do()
)

docs = query_result['data']['Get'][class_name]
print(f"Found {len(docs)} results")
for doc in docs:
    distance = doc['_additional']['distance']
    print("---------")
    print(distance)
    print(doc['text'])



In [ ]:

additional_clause = {
  "featureProjection": [
    "vector"
  ]
}
additional_setting = {
  "dimensions": 2
}

query_result = (
  client.query
  .get('Wiki_Node', "doc_id,extra_info,text")
  .with_near_vector({"vector": vector})
  .with_limit(5)
  .with_additional(
    (additional_clause, additional_setting)
  )
  .do()
)
print(query_result)

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from llama_index.legacy import LangchainEmbedding

embed_model = LangchainEmbedding(OpenAIEmbeddings())
vector = embed_model.get_query_embedding("How do I run MATLAB on the cluster?")


query_result = (
    client.query
    .get(class_name='Slack_Node', properties="text")
    .with_near_vector({"vector": vector})
    .with_limit(5)
    .do()
)

query_result

In [ ]:

query = "Singapore"
query_result = (
    client.query
    .get(class_name='Web_Node', properties="text")
    .with_bm25(query=query)
    .with_limit(1)
    .do()
)
print(query_result)
print()

query = "ipalm"
query_result = (
    client.query
    .get(class_name='Web_Node', properties="text,link,title")
    .with_bm25(query=query)
    .with_limit(10)
    .do()
)
results = query_result['data']['Get']['Web_Node']
print(f"{len(results)} results found")
for result in results:
    print(result)


In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from llama_index.legacy import LangchainEmbedding

query = "Who is from Singapore?"
embed_model = LangchainEmbedding(OpenAIEmbeddings())
vector = embed_model.get_query_embedding(query)
query_result = (
    client.query
    .get(class_name='Web_Node', properties="doc_id,text,title")
    .with_hybrid(query=query,vector=vector,alpha=0.50,properties=["text"])
    .with_limit(3)
    .do()
)

query_result

In [ ]:
query = "flatpak"
query_result = (
    client.query
    .get(class_name='Slack_Node', properties="text")
    .with_bm25(query=query)
    .with_limit(5)
    .do()
)

query_result

In [ ]:
client.schema.get("Web_Node")

In [ ]:
from datetime import datetime

query_result = (
    client.query
    .get(class_name='SurveyResponses', properties=["query","survey"],)
    .with_limit(5)
    .with_additional(['creationTimeUnix'])
    .do()
)

arr = query_result['data']['Get']['SurveyResponses']

for obj in arr:
    timestamp = int(obj['_additional']['creationTimeUnix']) / 1000.0
    date = datetime.fromtimestamp(timestamp)
    print(date.strftime("%Y-%m-%d %I:%M %p"))
    print(f"{obj['survey']} - {obj['query']}")


In [ ]:
# select object based on some criteria

where_filter = {
  "path": ["source"],
  "operator": "Equal",
  "valueString": "Slack"
}
query_result = (
  client.query
  .get(class_name='Janelia_Node', properties="doc_id,_node_content,text").with_additional(["id", "vector","distance"])
  .with_where(where_filter)
  .do()
)

query_result


In [ ]:
# delete objects based on some criteria

result = (
    client.batch.delete_objects(
        class_name='Janelia_Node',
        where={
            'path': ['source'],
            'operator': 'Equal',
            'valueText': 'Wiki'
        },
        dry_run=True,
        output='verbose'
    )
)

result

In [ ]:

query_result = (
  client.query
  .get(class_name='Slack_Node', properties="doc_id,channel,ts")
  #.with_additional(["id", "vector"])
  .with_additional(["id","creationTimeUnix","lastUpdateTimeUnix","vector"])
  .do()
)

query_result
